In [3]:
import pandas as pd
import geopandas as gpd
import sqlite3
from tqdm import tqdm
import numpy as np

import os
import zipfile

In [4]:
DIR_RAILWAY_JP = "Z:/lab/accessibility/Railway_Japan/"
TARGET_JCODE = "15209"

In [5]:
def zip_text_files(directory_path, output_zip_file):
    # 新しいZIPファイルを作成
    with zipfile.ZipFile(output_zip_file, 'w') as zipf:
        # 指定ディレクトリのファイルをループ
        for foldername, subfolders, filenames in os.walk(directory_path):
            for filename in filenames:
                if filename.endswith('.txt'):
                    # ファイルパスを取得
                    file_path = os.path.join(foldername, filename)
                    # ZIPファイルにファイルを追加
                    zipf.write(file_path, arcname=filename)
                    # print(f'Added {filename} to {output_zip_file}')

In [46]:
# jcodeの抽出
conn = sqlite3.connect("Z:\\lab\\accessibility\\Accessibility_Data.db")
cursor = conn.cursor()
cursor.execute("SELECT jcode FROM goverment")
jcodes = [r[0] for r in cursor.fetchall()]
conn.close()

In [47]:
# 日本全年国分のtrip抽出
conn = sqlite3.connect("C:/lab/accessibility/Railway_Japan/TripDataAllJapanRail.db")
cursor = conn.cursor()
# cursor.execute(f"select * from sqlite_master")
# for r in cursor.fetchall():
#     print(r)

cursor.execute(f"select * from stop_times")
stop_times = cursor.fetchall()
cursor.execute(f"select * from routes")
routes = cursor.fetchall()
cursor.execute(f"select * from trips")
trips = cursor.fetchall()
conn.close()

In [50]:
for TARGET_JCODE in tqdm(jcodes):

    DIR_OUT_GTFS    = "Z:\\lab\\accessibility\\Railway_Japan\\regional\\"
    Agency_id       = TARGET_JCODE +"-RAIL"
    if os.path.exists(DIR_OUT_GTFS+f"{Agency_id}.gtfs"):
        continue

    conn = sqlite3.connect(DIR_RAILWAY_JP + "StationPortDatabase.db")
    # station = pd.read_sql_query(sql="select * from stationHasGeom",con=conn)
    cursor = conn.cursor()
    cursor.execute(f"select * from stationHasGoverment WHERE jcode = '{TARGET_JCODE}'")
    Target_tation_Ids = []
    for r in cursor.fetchall():
        Target_tation_Ids.append(r[0])

    cursor.execute(f"select * from stationHasGeom")
    station_jp = cursor.fetchall()

    conn.close()


    if len(Target_tation_Ids) == 0:
        continue


    Target_Trip_Ids = []
    for stop_time in stop_times:
        if stop_time[3] in Target_tation_Ids:
            Target_Trip_Ids.append(stop_time[0])
    Target_Trip_Ids = set(Target_Trip_Ids)

    STOP_TIMES = []
    for stop_time in stop_times:
        if stop_time[0] in Target_Trip_Ids:
            stop_time = list(stop_time) 
            if stop_time[1] != "":
                stop_time[1] = stop_time[1]+":00"
            if stop_time[2] != "":
                stop_time[2] = stop_time[2]+":00"
            stop_time = tuple(stop_time) 
            STOP_TIMES.append(stop_time)
    

    if len(STOP_TIMES) == 0:
        continue
    


    TRIPS = []
    for trip in trips:
        if trip[0] in Target_Trip_Ids:
            TRIPS.append(trip)

    Route_Ids = []
    for trip in trips:
        if trip[0] in Target_Trip_Ids:
            Route_Ids.append(trip[1])
    Route_Ids = set(Route_Ids)
    ROUTES = []
    for route in routes:
        if route[0] in Route_Ids:
            ROUTES.append(route)

    Stop_Ids = []
    for stop_time in stop_times:
        if stop_time[0] in Target_Trip_Ids:
            Stop_Ids.append(stop_time[3])
    Stop_Ids = set(Stop_Ids)

    STOPS = []
    for stops in station_jp:
        if stops[0] in Stop_Ids:
            STOPS.append(stops)



    # GTFSデータの作成
    os.makedirs(DIR_OUT_GTFS+f"{Agency_id}.gtfs",exist_ok=True) # ディレクトリ作成
    Agency_name     = "Yawata Kyoto"
    Agency_url      = "https:/hoge/"
    Agency_timezone = "Asia/Tokyo"
    Agency_lang     = "ja"
    pd.DataFrame({"agency_id":[Agency_id],"agency_name":[Agency_name],"agency_url":[Agency_url],"agency_timezone":[Agency_timezone],"agency_lang":[Agency_lang]}).to_csv(DIR_OUT_GTFS+f"{Agency_id}.gtfs/agency.txt",index=False,sep=",")

    """calendar.txt"""
    Start_date = 20240401
    End_date = 20250401
    calendar_txt = {
        "service_id":["Weekday","Saturday","Holiday"],
        "monday":[1,0,0],
        "tuesday":[1,0,0],
        "wednesday":[1,0,0],
        "thursday":[1,0,0],
        "friday":[1,0,0],
        "saturday":[0,1,0],
        "sunday":[0,0,1],
        "start_date":[Start_date,Start_date,Start_date],
        "end_date":[End_date,End_date,End_date]
    }
    pd.DataFrame(calendar_txt).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/calendar.txt",index=False,sep=",")

    """stops.txt"""
    pd.DataFrame(np.array(STOPS)[:,[0,1,3,4]],columns=["stop_id","stop_name","stop_lat","stop_lon"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/stops.txt",index=False,sep=",")

    """routes.txt"""
    pd.DataFrame(ROUTES,columns=["route_id","route_short_name","route_type"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/routes.txt",index=False,sep=",")

    """trips.txt"""
    pd.DataFrame(TRIPS,columns=["trip_id","route_id","service_id","trip_short_name"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/trips.txt",index=False,sep=",")

    """stop_times.txt"""
    pd.DataFrame(STOP_TIMES,columns=["trip_id","arrival_time","departure_time","stop_id","stop_sequence"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/stop_times.txt",index=False,sep=",")

    """zip化"""
    # ディレクトリパスと出力するZIPファイルの名前
    directory_path  = DIR_OUT_GTFS+f"/{Agency_id}.gtfs"
    output_zip_file = DIR_OUT_GTFS+f"/{Agency_id}.gtfs.zip"
    # 関数を呼び出してZIP圧縮を実行
    zip_text_files(directory_path, output_zip_file)

 56%|█████▋    | 1073/1902 [9:22:11<7:14:21, 31.44s/it]


FileNotFoundError: [WinError 53] ネットワーク パスが見つかりません。: 'Z:\\'